In [ ]:
import pandas as pd
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch
import numpy as np
from constant import AFFECTNETPATH

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.set_default_tensor_type(torch.cuda.FloatTensor)

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, csv_file,base_path,transform=None):
        self.fields = ['subDirectory_filePath', 'expression','valence','arousal']
        self._table = pd.read_csv(csv_file,usecols=self.fields)
        
        self._table=self._table[self._table['expression'] <8]

        self._table=self._table.reset_index(drop=True)

        self._base_path=base_path
   



        self.transform=transforms.Compose([
                     transforms.Resize((64,64)),
                     transforms.ToTensor(),   
                     transforms.Normalize((0.5,0.5,0.5 ), (0.5, 0.5,0.5))])

    def __len__(self):
        return len(self._table)

    def __getitem__(self, idx):
        folder_name = os.path.join(self._base_path,self._table.subDirectory_filePath[idx])
        img=Image.open(folder_name)
        img=self.transform(img)
        valence = torch.from_numpy(np.array(self._table.valence[idx]))
        return (img,valence.float())
            
        

In [3]:
Affect_Train=ImageDataset('training.csv',AFFECTNETPATH)
Affect_Valid=ImageDataset('validation.csv',AFFECTNETPATH)

In [5]:
train_dataloader = DataLoader(Affect_Train, batch_size=256
                       , num_workers=0,shuffle=True)

valid_dataloader = DataLoader(Affect_Valid, batch_size=256
                       , num_workers=0)    

In [7]:
import torchvision.models as models
import torch.nn as nn
resnet18 = models.resnet18(pretrained=True)

conv1 is unfrozen
bn1 is unfrozen
relu is unfrozen
maxpool is unfrozen
layer1 is unfrozen
layer2 is unfrozen
layer3 is unfrozen
layer4 is unfrozen
avgpool is unfrozen
fc is unfrozen


In [8]:
from module import video_model
model=video_model.Face_Feature(resnet18)
model.to(device)

Face_Feature(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [10]:
loss_func = torch.nn.L1Loss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0005, betas=(0.5, 0.999))

In [ ]:
train_loss=[]
val_loss=[]
num_epochs=50

for epochs in (range(0,num_epochs)):
    model.train()
    print("Training Epoch: ", epochs+1,"\n")
   
    for i_batch, (img,valence) in tqdm(enumerate(train_dataloader)):
        batch_size=img.size(0)
        optimizer.zero_grad()
        img=img.to(device)
        val_output=model(img)
        loss=loss_func(val_output,valence.to(device))
        loss.backward()
        optimizer.step()
        if i_batch%500==0:
            print("Batch: ",i_batch+1,"/",len(train_dataloader))
            print("MSE loss: ", loss.item())
            train_loss.append(loss.item())
    model.eval()
    avg_loss=1e6
    temp_loss=0
    for i_batch, (img,valence) in enumerate(valid_dataloader):
        
        batch_size=img.size(0)

        img=img.to(device)
        val_output=model(img)
        loss1=loss_func(val_output,valence.to(device))
        temp_loss+=loss1.item()
    avg_temp_loss=    temp_loss/len(valid_dataloader)
    print("ValidationLoss: ",avg_temp_loss)
    val_loss.append(avg_temp_loss)
    if avg_temp_loss<avg_loss:
        avg_loss=avg_temp_loss
        torch.save(model,"face_feature.h")
        torch.save(model.resnet,"pre_trained_resnet18_face.h")
    

    

    
        

Training Epoch:  1 



/opt/conda/envs/torch/lib/python3.7/site-packages/torch/nn/modules/loss.py:88: UserWarning: Using a target size (torch.Size([256])) that is different to the input size (torch.Size([256, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Batch:  1 / 1124
MSE loss:  0.5328238606452942
Batch:  501 / 1124
MSE loss:  0.45904794335365295
Batch:  1001 / 1124
MSE loss:  0.45445358753204346


/opt/conda/envs/torch/lib/python3.7/site-packages/torch/nn/modules/loss.py:88: UserWarning: Using a target size (torch.Size([164])) that is different to the input size (torch.Size([164, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)



ValidationLoss:  0.5638815350830555
Training Epoch:  2 



/opt/conda/envs/torch/lib/python3.7/site-packages/torch/nn/modules/loss.py:88: UserWarning: Using a target size (torch.Size([160])) that is different to the input size (torch.Size([160, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/opt/conda/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Face_Feature. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ResNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for containe

Batch:  1 / 1124
MSE loss:  0.46809548139572144
Batch:  501 / 1124
MSE loss:  0.46355366706848145
Batch:  1001 / 1124
MSE loss:  0.44099533557891846

ValidationLoss:  0.5087571498006582
Training Epoch:  3 



Batch:  1 / 1124
MSE loss:  0.47007936239242554
Batch:  501 / 1124
MSE loss:  0.43787217140197754
Batch:  1001 / 1124
MSE loss:  0.4670810401439667

ValidationLoss:  0.5344849824905396
Training Epoch:  4 



Batch:  1 / 1124
MSE loss:  0.4602484405040741
Batch:  501 / 1124
MSE loss:  0.47904956340789795
Batch:  1001 / 1124
MSE loss:  0.4610998332500458

ValidationLoss:  0.5311878994107246
Training Epoch:  5 



Batch:  1 / 1124
MSE loss:  0.46678227186203003
